# Vector Database

## Demo code to store and search via a vector database
A quick demo code to store and search based on data from amazon-fine-food-reviews

Please ensure you have *.env* file in your HOME/Documents/src/openai/ folder

In [2]:
from openai import AzureOpenAI
from dotenv import load_dotenv
import pandas as pd
import tiktoken
import os

env_path = os.getenv("HOME") + "/Documents/src/openai/.env"
load_dotenv(dotenv_path=env_path, verbose=True)

os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-05-15"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://pvg-azure-openai-uk-south.openai.azure.com"

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version="2023-05-15"
)

In [3]:
db_path = "data/amazon-food-reviews-faiss"
input_path = "data/fine_food_reviews_1k.csv"

In [4]:
from langchain.embeddings import AzureOpenAIEmbeddings
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.vectorstores import FAISS


loader = CSVLoader(file_path=input_path, encoding="utf-8", csv_args={'delimiter': ','})
data = loader.load()

db = FAISS.from_documents(data, AzureOpenAIEmbeddings())

/Users/I069899/anaconda3/envs/py310/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.azure_openai.AzureOpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureOpenAIEmbeddings`.
  warn_deprecated(


In [5]:
db.save_local(db_path)

In [6]:
new_db = FAISS.load_local(db_path, AzureOpenAIEmbeddings())

In [9]:
query = "bad packaging"
answer_list = new_db.similarity_search(query)
for ans in answer_list:
    print(ans.page_content + "\n")

: 439
Time: 1351209600
ProductId: B0048KH218
UserId: A2JAEJJA6TJHLY
Score: 4
Summary: Busted
Text: The bag came broken. Product was leaking out of the box, due to poor packing standards.<br />Hope next items arrive unscathed. Quinoa tasted good.

: 340
Time: 1351123200
ProductId: B002CJAT6W
UserId: ANP2X414OHQNS
Score: 3
Summary: damaged can in group
Text: One of the cans arrived damaged with a collection of bugs in it.  The product itself is fine in the other cans, and I am using the rest.  My cats really like this food and I have ordered it before with no smashed cans.  I hope the company is not foisting old product on online shoppers.

: 1
Time: 1351123200
ProductId: B003JK537S
UserId: A3JBPC3WFUT5ZP
Score: 1
Summary: Arrived in pieces
Text: Not pleased at all. When I opened the box, most of the rings were broken in pieces. A total waste of money.

: 12
Time: 1351123200
ProductId: B000K8T3OQ
UserId: AK43Y4WT6FFR3
Score: 1
Summary: Broken in a million pieces
Text: Chips were broken i

In [ ]:
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.75}
)
docs = retriever.get_relevant_documents("bad packaging")
for doc in docs:
    print(doc.page_content + "\n")

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import AzureChatOpenAI

llm = AzureChatOpenAI(model_name="gpt-35-turbo", temperature=0.3)
qa_chain = RetrievalQA.from_chain_type(llm,
             retriever=new_db.as_retriever(search_type="similarity_score_threshold",
               search_kwargs={"score_threshold": 0.75}))
qa_chain.combine_documents_chain.verbose = True
qa_chain.return_source_documents = True

In [ ]:
qa_chain({"query": "bad packaging"})